## Configure a face library system

 Structure of our library system:

```library```<br/>
|__```celebrity_1``` <br/>
|   |   meta.npz
|    |   face_1.png
|    |   face_2.png
|    
|__```celeberity_2```
|
|
|
|___```(videoname)_(cluster no.)```
    

In [1]:
# Constants here
LIB_ROOT = '/home/ubuntu/visual_search_assistant/M3/library'
META_FNAME = 'meta.npz'

### Pre-processing the library
Extract encoding from faces

In [5]:
import face_recognition
import os
import numpy as np

def extract_encodings_dir(dir,mean=False):
    images = []
    names = []
    encodings = []
    
    for image_pth in os.listdir(dir):
        if image_pth.endswith('npz'):
            continue
        img = face_recognition.load_image_file(os.path.join(dir,image_pth))
        images.append(img)
        names.append(image_pth)
    
    for idx,(img,name) in enumerate(zip(images,names)):
        print('[%d / %d] Extracting encoding for %s' % (idx+1,len(images),name))
        embd = face_recognition.face_encodings(img)[0]
        encodings.append(embd)
    
    if mean:
        res = np.mean(encodings,axis=0)
        return res
    else:
        return encodings
    
    

### Pre-process the library

In [7]:
import os
import pickle

# preprocessing : convert single files into directories
identities = os.listdir(LIB_ROOT)
metadata_pth = os.path.join(LIB_ROOT,'meta.npz')
metadata = {
}
for fname in identities:
    src_pth = os.path.join(LIB_ROOT,fname)
    
    if not os.path.isdir(src_pth):
        
        name = src_pth.split('.')[0]
        dst_pth = os.path.join(LIB_ROOT,name)
        os.mkdir(dst_pth) # create directory for the identity
        os.rename(src_pth,os.path.join(dst_pth,fname)) # move single image file
        src_pth = dst_pth
        print("Created new directory for %s" % name)
    else:
        name = fname
        
    mean_encoding = extract_encodings_dir(src_pth,mean=True)
    metadata[fname] = {
        "encoding":mean_encoding,
        "name":'',
        "id":fname,
        "clusterlist":[fname]
    }
    
    print("Processing identity %s" % name)
with open(metadata_pth,'wb') as f:
    pickle.dump(metadata,f,protocol=pickle.HIGHEST_PROTOCOL)

[1 / 1] Extracting encoding for goora_kim.jpg
Processing identity goora_kim
[1 / 1] Extracting encoding for youngmi_ahn.jpeg
Processing identity youngmi_ahn
[1 / 1] Extracting encoding for radio_star.jpg
Processing identity radio_star
[1 / 1] Extracting encoding for Donna_Meagle.jpg
Processing identity Donna_Meagle
[1 / 1] Extracting encoding for leslie_knope.jpg
Processing identity leslie_knope
[1 / 1] Extracting encoding for gookjin_kim.jpeg
Processing identity gookjin_kim
--Return--
None
> /tmp/ipykernel_24449/1582884184.py(33)<module>()
     31     print("Processing identity %s" % name)
     32 #     import pdb;pdb.set_trace()
---> 33 import pdb;pdb.set_trace()
     34 with open(metadata_pth,'wb') as f:
     35     pickle.dump(metadata,f,protocol=pickle.HIGHEST_PROTOCOL)

ipdb> metadata.keys()
dict_keys(['goora_kim', 'youngmi_ahn', 'radio_star', 'Donna_Meagle', 'leslie_knope', 'gookjin_kim'])
ipdb> c


Check if correctly saved

In [9]:
example_meta_pth = '/home/ubuntu/visual_search_assistant/M3/library//meta.npz'
import pickle

with open(example_meta_pth,'rb') as rf:
    metadata = pickle.load(rf)

print(metadata.keys())
print(metadata['goora_kim'].keys())

dict_keys(['goora_kim', 'youngmi_ahn', 'radio_star', 'Donna_Meagle', 'leslie_knope', 'gookjin_kim'])
dict_keys(['encoding', 'name', 'id', 'clusterlist'])
